In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from bybit import *
from passivbot import calc_diff
import matplotlib.pyplot as plt
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'e'
settings = load_live_settings(exchange, user)
settings['exchange'] = exchange
s = settings['symbol']

In [ ]:
try:
    await bot.cc.close()
except:
    pass
bot = await create_bot(user, settings)
cc = bot.cc

In [ ]:
await bot.update_position()
bot.position

In [1]:
!pwd

/Users/e/Dropbox/pb_v2.0.0_beta


In [ ]:
n_days = 30

In [ ]:
await bot.init_my_trades(n_days)

In [ ]:
len(bot.my_trades)

In [ ]:
mtdf = pd.DataFrame(bot.my_trades)
le = mtdf[(mtdf.side == 'buy') & (mtdf.custom_id.str.startswith('entry'))]
se = mtdf[(mtdf.side == 'sell') & (mtdf.custom_id.str.startswith('entry'))]
lx = mtdf[(mtdf.side == 'sell') & (mtdf.custom_id.str.startswith('close'))]
sx = mtdf[(mtdf.side == 'buy') & (mtdf.custom_id.str.startswith('close'))]
ss = mtdf[mtdf.custom_id.str.startswith('stop')]

le.price.plot(style='b.-')
lx.price.plot(style='ro')
ss.price.plot(style='gx')

In [ ]:
se.price.plot(style='r.-')
sx.price.plot(style='bo')

In [ ]:
mtdf.tail(20)

In [ ]:
mtdf.tail(60)